In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import numpy as np

import data_tool as dt

from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Clear any logs from previous runs
%rm -rf ./temp5/logs/

In [4]:
class NMSE_Accuracy(keras.metrics.Metric):
    def __init__(self, name="nmse_accuracy", **kwargs):
        super(NMSE_Accuracy, self).__init__(name=name, **kwargs)
        self.sum_nmse = self.add_weight(name="sum_nmse", initializer="zeros")
        self.sample_number = self.add_weight(name="sample_number", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        diff = y_pred - y_true
        fenmu = tf.reduce_sum(tf.pow(y_true, 2), 1)
        current_nmse = tf.reduce_sum(tf.reduce_sum(tf.pow(diff, 2), 1) / fenmu)
        current_nmse = tf.where(tf.math.is_nan(current_nmse)|tf.math.is_inf(current_nmse), tf.zeros_like(current_nmse), current_nmse)
        current_nmse = tf.cast(current_nmse, "float32")
        self.sum_nmse.assign_add(current_nmse)
#         current_sample_number = y_true.shape[0]
        current_sample_number = tf.math.count_nonzero(fenmu)
        current_sample_number = tf.cast(current_sample_number, "float32")
        self.sample_number.assign_add(current_sample_number)

    def result(self):
        return 10 * (tf.math.log(self.sum_nmse / self.sample_number) / tf.math.log(10.))

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.sum_nmse.assign(0.0)
        self.sample_number.assign(0.0)




In [6]:
user_number = 100
code_length = 50
epoch_number = 30
batch_size = 1000

estimator = keras.Sequential(
    [
        tf.keras.layers.BatchNormalization(name="bp1"),
        layers.Dense(user_number + code_length, activation="relu", name="layer1"),
        tf.keras.layers.BatchNormalization(name="bp2"),
        layers.Dense(user_number + code_length, activation="relu", name="layer2"),
        tf.keras.layers.BatchNormalization(name="bp3"),
        layers.Dense(user_number + code_length, activation="relu", name="layer3"),
        tf.keras.layers.BatchNormalization(name="pb4"),
        layers.Dense(user_number, activation="relu", name="layer4"),
    ],name = 'estimator'
)
# Call model on a test input
y = tf.ones((3, code_length))
x_hat = estimator(y)
estimator.summary()

signature_matrix = dt.get_random_binary_signature_matrix(user_number, code_length)
y_set, x_set, h_set = dt.get_dataset(1000000, signature_matrix, is_fading=1)
y_set = y_set.astype("float32")
x_set = x_set.astype("float32")
h_set = h_set.astype("float32")
y_test, x_test, h_test = dt.get_dataset(10000, signature_matrix, is_fading=1)
y_test = y_test.astype("float32")
x_test = x_test.astype("float32")
h_test = h_test.astype("float32")

Model: "estimator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bp1 (BatchNormalization)     (3, 50)                   200       
_________________________________________________________________
layer1 (Dense)               (3, 150)                  7650      
_________________________________________________________________
bp2 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer2 (Dense)               (3, 150)                  22650     
_________________________________________________________________
bp3 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer3 (Dense)               (3, 150)                  22650     
_________________________________________________________________
pb4 (BatchNormalization)     (3, 150)                  60

In [8]:
estimator.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.RMSprop(),
    metrics=NMSE_Accuracy(),
)

log_dir="./temp5/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = estimator.fit(y_set, h_set, batch_size=1000, epochs=30, validation_split=0.2, callbacks=[tensorboard_callback])

test_scores = estimator.evaluate(y_test, h_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/30
800/800 [==============================] - 14s 15ms/step - loss: 0.1098 - nmse_accuracy: -3.3543 - val_loss: 0.0201 - val_nmse_accuracy: -10.5899
Epoch 2/30
800/800 [==============================] - 10s 12ms/step - loss: 0.0148 - nmse_accuracy: -11.9557 - val_loss: 0.0106 - val_nmse_accuracy: -13.4975
Epoch 3/30
800/800 [==============================] - 9s 11ms/step - loss: 0.0100 - nmse_accuracy: -13.7115 - val_loss: 0.0088 - val_nmse_accuracy: -14.3547
Epoch 4/30
800/800 [==============================] - 9s 11ms/step - loss: 0.0088 - nmse_accuracy: -14.3599 - val_loss: 0.0082 - val_nmse_accuracy: -14.7072
Epoch 5/30
800/800 [==============================] - 9s 11ms/step - loss: 0.0081 - nmse_accuracy: -14.7402 - val_loss: 0.0077 - val_nmse_accuracy: -15.0136
Epoch 6/30
800/800 [==============================] - 11s 14ms/step - loss: 0.0077 - nmse_accuracy: -14.8988 - val_loss: 0.0073 - val_nmse_accuracy: -15.2051
Epoch 7/30
800/800 [==============================] - 10

In [9]:
%tensorboard --logdir ./temp5/logs/fit